In [11]:
import datetime
import pandas as pd
import hopsworks
import os
from dotenv import load_dotenv
from functions import utils
import json

import datetime
import requests
import pandas as pd
import hopsworks
import datetime
from pathlib import Path
from functions import utils
import json
import re
import os
import warnings
warnings.filterwarnings("ignore")

In [12]:
load_dotenv()
HOPSWORKS_API_KEY = os.getenv("HOPSWORKS_API_KEY")

os.environ["HOPSWORKS_API_KEY"] = HOPSWORKS_API_KEY
print(HOPSWORKS_API_KEY)

AhdA5UndBaVILBvJ.cvcBxXmN8mo8HPjR1Ryb7XncvNQ6enGsnTHL3LZIUJJ75mpRD0tTptadPTgx4zRu


In [13]:
CITY = "dublin"


## Read the bike data from csv-files

In [14]:
# get directory path and all files in it
df_bike = pd.DataFrame()
for file in os.listdir(CITY):
    if file.endswith(".csv"):
        df_tmp = pd.read_csv(f"{CITY}/" + file,  parse_dates=['datetime'], skipinitialspace=True)
        df_tmp['station'] = file.split(".")[0]
        df_bike = pd.concat([df_bike, df_tmp])

df_bike = df_bike.set_index(['datetime', 'station']).reset_index(drop=False)

df_bike

,datetime,station,num_bikes_available
0,2025-01-06 15:06:11+00:00,AVONDALE_ROAD,19
1,2025-01-06 14:48:56+00:00,AVONDALE_ROAD,18
2,2025-01-06 13:49:04+00:00,AVONDALE_ROAD,18
3,2025-01-06 12:46:44+00:00,AVONDALE_ROAD,18
4,2025-01-06 11:49:15+00:00,AVONDALE_ROAD,11
...,...,...,...
1056433,2023-08-05 16:54:54+00:00,YORK_STREET_WEST,20
1056434,2023-08-05 17:56:37+02:00,YORK_STREET_WEST,16
1056435,2023-08-05 14:51:22+00:00,YORK_STREET_WEST,19
1056436,2023-08-05 15:54:31+02:00,YORK_STREET_WEST,17


In [15]:
df_bike.dropna(inplace=True)
df_bike["num_bikes_available"] = df_bike["num_bikes_available"].astype("float32")
df_bike


,datetime,station,num_bikes_available
0,2025-01-06 15:06:11+00:00,AVONDALE_ROAD,19.0
1,2025-01-06 14:48:56+00:00,AVONDALE_ROAD,18.0
2,2025-01-06 13:49:04+00:00,AVONDALE_ROAD,18.0
3,2025-01-06 12:46:44+00:00,AVONDALE_ROAD,18.0
4,2025-01-06 11:49:15+00:00,AVONDALE_ROAD,11.0
...,...,...,...
1056433,2023-08-05 16:54:54+00:00,YORK_STREET_WEST,20.0
1056434,2023-08-05 17:56:37+02:00,YORK_STREET_WEST,16.0
1056435,2023-08-05 14:51:22+00:00,YORK_STREET_WEST,19.0
1056436,2023-08-05 15:54:31+02:00,YORK_STREET_WEST,17.0


## Get the weather data from Open Mateo

First, we need to get the earliest date of the bike data so that we can get the weater data from Open Mateo for the same period.

In [16]:
earliest_bike_date_raw = pd.Series.min(df_bike['datetime'])
earliest_bike_date = earliest_bike_date_raw.strftime('%Y-%m-%d')
earliest_bike_datetime = earliest_bike_date_raw.strftime('%Y-%m-%d %H:%M:%S %Z')
last_bike_date_raw = pd.Series.max(df_bike['datetime'])
last_bike_date = last_bike_date_raw.strftime('%Y-%m-%d')
last_bike_datetime = last_bike_date_raw.strftime('%Y-%m-%d %H:%M:%S %Z')

earliest_bike_datetime, last_bike_datetime

('2023-08-05 12:56:53 UTC', '2025-01-06 15:06:11 UTC')

Then, we get the weather data from Open Mateo

In [17]:
weather_df = utils.get_historical_weather(CITY, earliest_bike_date, last_bike_date)
weather_df = weather_df.rename(columns={'date_x': 'datetime'})
weather_df.dropna(inplace=True)

# remove col date_y, date_only
weather_df = weather_df.drop(columns=['date_y', 'date_only'])

weather_df

Coordinates 53.32161331176758°N -6.501922607421875°E
Elevation 11.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s


,datetime,temperature_2m,apparent_temperature,rain,snowfall,wind_speed_10m,daylight_duration,rain_sum,city
0,2023-08-05 00:00:00+00:00,12.889501,11.347359,0.5,0.00,13.202726,55572.929688,26.800001,dublin
1,2023-08-05 01:00:00+00:00,12.589500,10.415937,4.7,0.00,17.114204,55572.929688,26.800001,dublin
2,2023-08-05 02:00:00+00:00,12.339500,9.593327,4.4,0.00,20.833395,55572.929688,26.800001,dublin
3,2023-08-05 03:00:00+00:00,12.589500,9.265682,2.9,0.00,25.420181,55572.929688,26.800001,dublin
4,2023-08-05 04:00:00+00:00,12.839500,9.678358,2.6,0.00,25.264202,55572.929688,26.800001,dublin
...,...,...,...,...,...,...,...,...,...
12451,2025-01-04 19:00:00+00:00,2.689500,-0.908888,0.5,0.00,12.108261,27707.722656,3.700000,dublin
12452,2025-01-04 20:00:00+00:00,2.639500,-1.459750,0.9,0.00,15.542662,27707.722656,3.700000,dublin
12453,2025-01-04 21:00:00+00:00,2.189500,-2.205879,0.8,0.07,17.337091,27707.722656,3.700000,dublin
12454,2025-01-04 22:00:00+00:00,1.689500,-3.011362,0.4,0.56,19.083395,27707.722656,3.700000,dublin


## Data validation rules

First, we validate that the number of bikes avaliable is always greater than or equal to zero.

In [18]:
if 'num_bikes_available' not in df_bike.columns:
    raise ValueError("The column 'num_bikes_available' does not exist in the dataframe.")

invalid_rows = df_bike[(df_bike['num_bikes_available'] < 0) | (df_bike['num_bikes_available'] > 100)]

if not invalid_rows.empty:
    print("Validation failed: Some rows have invalid values in 'num_bikes_available'.")
    print(invalid_rows)
    assert False
else:
    print("Validation passed: All values in 'num_bikes_available' are within the range 0 to 100.")
    assert True

Validation passed: All values in 'num_bikes_available' are within the range 0 to 100.


Then, we validate that the rainfall, snowfall, and wind speed are always greater than or equal to zero, and below 1000.

In [19]:
if 'rain' not in weather_df.columns:
    raise ValueError("The column 'rain' does not exist in the dataframe.")
if 'snowfall' not in weather_df.columns:
    raise ValueError("The column 'snowfall' does not exist in the dataframe.")
if 'wind_speed_10m' not in weather_df.columns:
    raise ValueError("The column 'wind_speed_10m' does not exist in the dataframe.")

invalid_rows = weather_df[(weather_df['rain'] < 0) | (weather_df['rain'] > 1000) |
                          (weather_df['snowfall'] < 0) | (weather_df['snowfall'] > 1000) |
                          (weather_df['wind_speed_10m'] < 0) | (weather_df['wind_speed_10m'] > 1000)]

if not invalid_rows.empty:
    print("Validation failed: Some rows have invalid values in the specified columns.")
    print(invalid_rows)
    assert False
else:
    print("Validation passed: All values in the specified columns are within the range 0 to 1000.")
    assert True


Validation passed: All values in the specified columns are within the range 0 to 1000.


## Connect to Hopsworks and create two feature groups (+more)

In [20]:
project = hopsworks.login()
fs = project.get_feature_store()

print(hopsworks.__version__)

Connection closed.
2025-01-07 12:17:20,177 INFO: Initializing external client
2025-01-07 12:17:20,178 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-01-07 12:17:21,538 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1207494
4.1.4


Save the city location and times for the data to Hopsworks for easy access later.

In [21]:
"""
latitude, longitude = utils.get_city_coordinates(CITY)

city_secrets_dict = {
    "city": CITY,
    "latitude": latitude,
    "longitude": longitude,
}

time_secrets_dict = {
    "earliest_bike_datetime": earliest_bike_datetime,
    "last_bike_datetime": last_bike_datetime
}

city_secrets_json = json.dumps(city_secrets_dict)
time_secrets_json = json.dumps(time_secrets_dict)

try:
    secrets.create_secret("city_secrets", city_secrets_json)
except hopsworks.RestAPIError:
    print("city_secrets already exists. To update, delete the secret in the UI (https://c.app.hopsworks.ai/account/secrets) and re-run this cell.")
    existing_key = secrets.get_secret("city_secrets").value
    print(f"{existing_key}")
    print()

try:
    secrets.create_secret("time_secrets", time_secrets_json)
except hopsworks.RestAPIError:
    print("time_secrets already exists. To update, delete the secret in the UI (https://c.app.hopsworks.ai/account/secrets) and re-run this cell.")
    existing_key = secrets.get_secret("time_secrets").value
    print(f"{existing_key}")
"""


'\nlatitude, longitude = utils.get_city_coordinates(CITY)\n\ncity_secrets_dict = {\n    "city": CITY,\n    "latitude": latitude,\n    "longitude": longitude,\n}\n\ntime_secrets_dict = {\n    "earliest_bike_datetime": earliest_bike_datetime,\n    "last_bike_datetime": last_bike_datetime\n}\n\ncity_secrets_json = json.dumps(city_secrets_dict)\ntime_secrets_json = json.dumps(time_secrets_dict)\n\ntry:\n    secrets.create_secret("city_secrets", city_secrets_json)\nexcept hopsworks.RestAPIError:\n    print("city_secrets already exists. To update, delete the secret in the UI (https://c.app.hopsworks.ai/account/secrets) and re-run this cell.")\n    existing_key = secrets.get_secret("city_secrets").value\n    print(f"{existing_key}")\n    print()\n\ntry:\n    secrets.create_secret("time_secrets", time_secrets_json)\nexcept hopsworks.RestAPIError:\n    print("time_secrets already exists. To update, delete the secret in the UI (https://c.app.hopsworks.ai/account/secrets) and re-run this cell.")\

### Feature group 1: Bike data

In [22]:
bike_fg = fs.get_or_create_feature_group(
    name='bike_data',
    description='How many bikes are available at each station at a given time',
    version=1,
    primary_key=["datetime", "station"],
    event_time="datetime"
)

bike_fg.insert(df_bike)

bike_fg.update_feature_description("num_bikes_available", "Number of bikes available at the station")
bike_fg.update_feature_description("datetime", "Timestamp of the observation")
bike_fg.update_feature_description("station", "Name of the station")

Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/1207494/fs/1195126/fg/1393695


Uploading Dataframe: 100.00% |██████████| Rows 1056438/1056438 | Elapsed Time: 02:50 | Remaining Time: 00:00


Launching job: bike_data_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1207494/jobs/named/bike_data_1_offline_fg_materialization/executions


### Feature group 2: Weather data

In [23]:
weather_fg = fs.get_or_create_feature_group(
    name='weather_data',
    description='Weather data for the city',
    version=1,
    primary_key=["datetime"],
    event_time="datetime"
)

weather_fg.insert(weather_df)

# "hourly": ["temperature_2m", "apparent_temperature", "rain", "snowfall", "snow_depth", "wind_speed_10m"],
# "daily": ["sunshine_duration", "daylight_duration", "rain_sum"]

weather_fg.update_feature_description("datetime", "Timestamp of the observation")
weather_fg.update_feature_description("temperature_2m", "Temperature in Celsius")
weather_fg.update_feature_description("apparent_temperature", "Apparent temperature in Celsius")
weather_fg.update_feature_description("rain", "Rain percipitation in mm")
weather_fg.update_feature_description("snowfall", "Snowfall in mm")
weather_fg.update_feature_description("wind_speed_10m", "Wind speed 10 m above ground level")
weather_fg.update_feature_description("daylight_duration", "Daylight duration in seconds")
weather_fg.update_feature_description("rain_sum", "Total rain in mm")

Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/1207494/fs/1195126/fg/1393696


Uploading Dataframe: 100.00% |██████████| Rows 12456/12456 | Elapsed Time: 00:02 | Remaining Time: 00:00


Launching job: weather_data_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1207494/jobs/named/weather_data_1_offline_fg_materialization/executions
